In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import nibabel as nib
import pickle
import cortex
%matplotlib inline

# Visualization on freesurfer brain
Run the below cell if all you want to do is to locally run the contents of paulscotti.github.io/EduCortex

In [ ]:
subject = 'fsaverage'
anat = cortex.db.get_anat(subject, 'raw')
anat_data = anat.get_data()
anat_vol = cortex.Volume.empty(subject, 'atlas_2mm')

cortex.webgl.show(data=anat_vol, template='my_template.html', labels_visible=(""), recache=True)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 26965


# Visualization of brain: Top 3 PCA of Neurosynth

In [6]:
comps = np.load('data/components.npz')['arr_0']
comps = comps.transpose(0,3,2,1)
comps.reshape(10,-1).shape

(10, 902629)

In [7]:
def make_colorvol(proj, meanstds, rgbpcs, mask, flips=tuple(), clip_lim=3):
    rgbdata = proj[rgbpcs,:]
    for f in flips:
        rgbdata[f] *= -1
    zrgb = (rgbdata.T / meanstds).T
    #zrgb = npp.rs(rgbdata.T).T
    crgb = np.clip(zrgb, -clip_lim, clip_lim) ## clip to 3 standard deviations
    srgb = crgb/clip_lim/2.0 + 0.5 ## scale to 0..1
    
    rgbvol = np.zeros([3]+list(mask.shape))
    rgbvol[0] = vox_to_mask(srgb[0], mask)
    rgbvol[1] = vox_to_mask(srgb[1], mask)
    rgbvol[2] = vox_to_mask(srgb[2], mask)

    colorvol = (rgbvol*255).astype(np.uint8)
    return colorvol.transpose(1,2,3,0)

def vox_to_mask(data, mask):
    dvol = mask.copy().astype(np.float)
    dvol[mask>0] = data
    return dvol

In [8]:
rescale = lambda x: x/x.std()

In [9]:
flat_comps = comps.reshape(10,-1)

colorvol = make_colorvol(flat_comps, flat_comps.std(1)[:3], [0,1,2], np.ones(comps[0].shape), clip_lim=4)

In [10]:
rgbvol = cortex.dataset.normalize((colorvol, 'fsaverage', 'atlas_2mm'))
rgbvol.alpha.vmin = 0

In [11]:
# cortex.webgl.show(data=rgbvol, template='my_template.html', labels_visible=(""), recache=True)

# to make your own localhost server:
cortex.webgl.make_static("static",data=rgbvol, template='my_template.html', labels_visible=(""), recache=True)

Generating new ctm file...
wm
wm
inflated
inflated


# Individual PCAs

In [69]:
pca = PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False).fit(concat_maps)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)

[0.10724451 0.07186315 0.04695774 0.03718591 0.03011859 0.02566107
 0.01888798 0.01386358 0.01315753 0.01177422]
[7371.67572625 6034.36808339 4877.89306864 4340.78108748 3906.57327303
 3605.9198214  3093.65168397 2650.43018536 2582.05702134 2442.55708149]


In [80]:
components = np.array([dataset.masker.unmask(pca.components_[i],output='array') for i in range(10)])
np.savez('components',components)

In [71]:
components = np.load("components.npz")
components

In [72]:
term_weights = pca.transform(concat_maps)

In [160]:
for i in range(10):
    a = term_weights[:,i].argsort()[::-1];
    print(np.concatenate((kept_terms.iloc[a[:5]], kept_terms.iloc[a[-5:]]), axis=None));
    print(i);

['motor' 'premotor' 'movements' 'movement' 'hand' 'medial' 'hippocampus'
 'medial prefrontal' 'emotional' 'amygdala']
0
['temporal' 'semantic' 'language' 'words' 'visual' 'striatum' 'insula'
 'somatosensory' 'motor' 'pain']
1
['auditory' 'superior temporal' 'speech' 'auditory cortex' 'sounds'
 'retrieval' 'task' 'prefrontal' 'memory' 'parietal']
2
['occipital' 'lateral occipital' 'visual' 'motion' 'intraparietal'
 'autobiographical' 'semantic' 'insula' 'auditory' 'temporal']
3
['hippocampus' 'amygdala' 'fusiform' 'faces' 'medial temporal'
 'inferior frontal' 'frontal gyrus' 'working memory' 'dorsolateral'
 'frontal']
4
['default' 'default mode' 'resting' 'resting state' 'dmn' 'nucleus'
 'insula' 'amygdala' 'striatum' 'reward']
5
['cerebellar' 'vi' 'cerebellum' 'hippocampal' 'hippocampus' 'insula'
 'medial prefrontal' 'junction' 'theory mind' 'social']
6
['auditory' 'auditory cortex' 'sounds' 'planum' 'planum temporale'
 'inferior frontal' 'words' 'semantic' 'reading' 'word']
7
['striat

## Anatomical labels

In [8]:
# load glasser atlas
lh_aparc_file = os.path.join('atlases','lh.HCP-MMP1.annot')
rh_aparc_file = os.path.join('atlases','rh.HCP-MMP1.annot')
lpinds, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file)
lpinds_orig, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file, True)
lpinds[lpinds_orig==0] = -1
rpinds, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file)
rpinds_orig, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file, True)
rpinds[rpinds_orig==0] = -1

aparc = cortex.Vertex(np.hstack([lpinds, rpinds]),'fsaverage')

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 9820


<JS: window.viewer>